In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_21240/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_21240/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_22_2,0.999672,0.830720,0.818349,0.999728,0.001367,1.131975,0.649756,0.001146,0.087913,0.036968,1.021012,0.037550,143.190825,284.525997,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_22_6,0.999682,0.830454,0.818102,0.999714,0.001324,1.133756,0.650638,0.001203,0.089049,0.036387,1.020358,0.036960,143.254129,284.589302,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_17_6,0.999444,0.830173,0.816157,0.994489,0.002314,1.135633,0.657597,0.009129,0.140536,0.048103,1.035576,0.048860,142.137684,283.472857,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_17_2,0.999511,0.829984,0.817542,0.995632,0.002037,1.136900,0.652640,0.007236,0.131525,0.045131,1.031316,0.045842,142.392744,283.727917,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_22_9,0.999681,0.829983,0.817726,0.999663,0.001329,1.136904,0.651982,0.001420,0.094895,0.036450,1.020427,0.037024,143.247310,284.582482,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,model_1_3_7,0.940835,0.786400,0.771217,0.939734,0.246277,1.428342,0.818342,0.331314,0.857090,0.496263,4.786583,0.504079,132.802593,274.137766,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
139,model_1_3_3,0.901314,0.768269,0.696619,0.901820,0.410784,1.549584,1.085177,0.539746,1.134008,0.640925,7.315923,0.651019,131.779374,273.114546,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
140,model_1_3_0,0.893280,0.764609,0.686784,0.894534,0.444224,1.574062,1.120355,0.579806,1.182494,0.666501,7.830069,0.676998,131.622852,272.958025,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
141,model_1_2_1,0.884416,0.754893,0.691027,0.883282,0.481122,1.639028,1.105178,0.408619,1.167433,0.693630,8.397387,0.704554,131.463268,272.798440,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
